In [2]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define emotions in RAVDESS dataset
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',                                                                
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}


In [4]:
# Function to extract features from audio
def extract_features(file_path, max_pad_len=174):
    try:
        audio, sample_rate = librosa.load(file_path, sr=22050)
        # Extract MFCCs
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        # Pad or truncate MFCCs to fixed length
        pad_width = max_pad_len - mfccs.shape[1]
        if pad_width > 0:
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfccs = mfccs[:, :max_pad_len]
        return mfccs
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Load RAVDESS dataset
def load_data(data_path):
    X, y = [], []
    for root, _, files in os.walk(data_path):
        for file in files:
            if file.endswith('.wav'):
                file_path = os.path.join(root, file)
                # Extract emotion from filename (e.g., 03-01-01-01-01-01-01.wav)
                emotion_code = file.split('-')[2]
                if emotion_code in emotions:
                    features = extract_features(file_path)
                    if features is not None:
                        X.append(features)
                        y.append(emotions[emotion_code])
    return np.array(X), np.array(y)


In [5]:
# Path to RAVDESS dataset (update this to your dataset path)
data_path = 'D:\dataset_speech\dataset'  # Replace with actual path to RAVDESS dataset

# Load and preprocess data
X, y = load_data(data_path)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Reshape X for CNN (add channel dimension)
X = X[..., np.newaxis]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)


In [6]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(40, 174, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(emotions), activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

# Train model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

d:\dataset_speech\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 38, 172, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 19, 86, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 17, 84, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 42, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 6, 40, 128)     │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 20, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 7680)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       983,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,076,872 (4.11 MB)

 Trainable params: 1,076,872 (4.11 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.1693 - loss: 4.1188 - val_accuracy: 0.3385 - val_loss: 1.8003
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.2658 - loss: 1.8454 - val_accuracy: 0.4375 - val_loss: 1.5780
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.3422 - loss: 1.6718 - val_accuracy: 0.4792 - val_loss: 1.4312
Epoch 4/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.4002 - loss: 1.5175 - val_accuracy: 0.5365 - val_loss: 1.2432
Epoch 5/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.4520 - loss: 1.4013 - val_accuracy: 0.5816 - val_loss: 1.1632
Epoch 6/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.5125 - loss: 1.2525 - val_accuracy: 0.6753 - val_loss: 0.9903
Epoch 7/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.5420 - loss: 1.1544 - val_accuracy: 0.7049 - val_loss: 0.9207
Epoch 8/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - accuracy: 0.5996 - loss: 1.0561 - val_accuracy: 0.7552 - v

In [7]:
# Evaluate model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_accuracy:.4f}")

# Save model
model.save('emotion_speech_model.keras')

# Function to predict emotion from new audio
def predict_emotion(file_path):
    features = extract_features(file_path)
    if features is None:
        return "Error processing audio"
    features = features[np.newaxis, ..., np.newaxis]
    prediction = model.predict(features)
    predicted_emotion = label_encoder.inverse_transform([np.argmax(prediction)])[0]
    return predicted_emotion

Test accuracy: 0.9444


In [20]:

if __name__ == "__main__":
    # Define path to a specific test .wav file
    test_file = r'D:\dataset_speech\dataset\testcase\laughing-man-27517.mp3'  # Replace with an actual file path

    # Ensure the file exists
    if os.path.isfile(test_file):
        predicted_emotion = predict_emotion(test_file)
        print(f"Predicted emotion: {predicted_emotion}")
    else:
        print(f"File not found: {test_file}")

    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted emotion: disgust
